In [1]:
import tensorflow as tf
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-02-01 23:18:49.061077: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-01 23:18:50.006896: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22345 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-02-01 23:18:50.008073: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22345 MB memory:  -> device: 1, name: Graphics Device, pci bus id: 0000:65:00.0, compute capability: 8.6


In [2]:
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

Number of devices: 2


In [3]:
#flag들 정의
model_name = 'efficientdet-d0'
logdir='/tmp/deff/'
runmode='dry'
trace_filename= None
threads= 0
bm_runs=10
tensorrt= None
delete_logdir=True
freeze=False
use_xla=False
batch_size=1
ckpt_path=None
export_ckpt= None
hparams=''
input_image=None
output_image_dir=None
input_video=None
output_video=None
line_thickness=None
max_boxes_to_draw=100
min_score_thresh=0.4
nms_method='hard'
saved_model_dir='/tmp/saved_model'
tflite_path=None

In [4]:
import os
MODEL = 'efficientdet-d0'
ckpt_path = os.path.join(os.getcwd(), MODEL)
print('Use model in {}'.format(ckpt_path))

img_path = os.path.join(os.getcwd(), 'p_dataset/img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

from PIL import Image
# Get the largest of height/width and round to 128.
image_size = max(Image.open(img_path).size)

Use model in /home/gmpark/automl/efficientdet/efficientdet-d0


In [5]:
runmode='saved_model'
model_name=MODEL
ckpt_path=ckpt_path
hparams="image_size=1920x1280"
saved_model_dir = 'savedmodel'

In [6]:
#분산 선언
from model_inspect import ModelInspector
with mirrored_strategy.scope():
    inspector = ModelInspector(
    model_name=model_name,
    logdir=logdir,
    tensorrt=tensorrt,
    use_xla=use_xla,
    ckpt_path=ckpt_path,
    export_ckpt=export_ckpt,
    saved_model_dir=saved_model_dir,
    tflite_path=tflite_path,
    batch_size=batch_size,
    hparams=hparams,
    score_thresh=min_score_thresh,
    max_output_size=max_boxes_to_draw,
    nms_method=nms_method)
    
    

In [7]:

def train_step(inputs):
    runmode,input_image,output_image_dir,input_video,output_video,line_thickness,max_boxes_to_draw,min_score_thresh,nms_method,bm_runs,threads,trace_filename = inputs
    print(runmode,input_image,output_image_dir,input_video,output_video,line_thickness,max_boxes_to_draw,min_score_thresh,nms_method,bm_runs,threads,trace_filename)
    inspector.run_model(
      runmode,
      input_image=input_image,
      output_image_dir=output_image_dir,
      input_video=input_video,
      output_video=output_video,
      line_thickness=line_thickness,
      max_boxes_to_draw=max_boxes_to_draw,
      min_score_thresh=min_score_thresh,
      nms_method=nms_method,
      bm_runs=bm_runs,
      threads=threads,
      trace_filename=trace_filename)

In [8]:
@tf.function
def distributed_train_step():
    
    mirrored_strategy.run(train_step,args=(args,))
args=(runmode,input_image,output_image_dir,input_video,output_video,line_thickness,max_boxes_to_draw,min_score_thresh,nms_method,bm_runs,threads,trace_filename)
distributed_train_step()

saved_model None None None None 2 200 0.35 hard 10 0 None


2022-02-01 23:18:52.143648: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22345 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-02-01 23:18:52.144468: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22345 MB memory:  -> device: 1, name: Graphics Device, pci bus id: 0000:65:00.0, compute capability: 8.6


/home/gmpark/automl/efficientdet/utils.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  from tensorflow.python.tpu import tpu_function  # pylint:disable=g-direct-tensorflow-import
/home/gmpark/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:


INFO:tensorflow:Error reported to Coordinator: in user code:

    File "/tmp/ipykernel_1294/105862541.py", line 4, in train_step  *
        inspector.run_model(
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 462, in run_model  *
        self.export_saved_model(**config_dict)
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 150, in export_saved_model  *
        driver.build()
    File "/home/gmpark/automl/efficientdet/inference.py", line 465, in build  *
        restore_ckpt(
    File "/home/gmpark/automl/efficientdet/inference.py", line 210, in restore_ckpt  *
        var_dict = ema.variables_to_restore(ema_vars)

    TypeError: Variable is unhashable. Instead, use variable.ref() as the key. (Variable: MirroredVariable:{
      0: <tf.Variable 'Variable:0' shape=() dtype=int64>,
      1: <tf.Variable 'Variable/replica_1:0' shape=() dtype=int64>
    })
Traceback (most recent call last):
  File "/home/gmpark/anaconda3/envs/ed/lib/python3.7/site-packag

INFO:tensorflow:Error reported to Coordinator: in user code:

    File "/tmp/ipykernel_1294/105862541.py", line 4, in train_step  *
        inspector.run_model(
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 462, in run_model  *
        self.export_saved_model(**config_dict)
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 150, in export_saved_model  *
        driver.build()
    File "/home/gmpark/automl/efficientdet/inference.py", line 465, in build  *
        restore_ckpt(
    File "/home/gmpark/automl/efficientdet/inference.py", line 210, in restore_ckpt  *
        var_dict = ema.variables_to_restore(ema_vars)

    TypeError: Variable is unhashable. Instead, use variable.ref() as the key. (Variable: MirroredVariable:{
      0: <tf.Variable 'Variable:0' shape=() dtype=int64>,
      1: <tf.Variable 'Variable/replica_1:0' shape=() dtype=int64>
    })
Traceback (most recent call last):
  File "/home/gmpark/anaconda3/envs/ed/lib/python3.7/site-packag

TypeError: in user code:

    File "/tmp/ipykernel_1294/4115394738.py", line 4, in distributed_train_step  *
        mirrored_strategy.run(train_step,args=(args,))
    File "/tmp/ipykernel_1294/105862541.py", line 4, in train_step  *
        inspector.run_model(
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 462, in run_model  *
        self.export_saved_model(**config_dict)
    File "/home/gmpark/automl/efficientdet/model_inspect.py", line 150, in export_saved_model  *
        driver.build()
    File "/home/gmpark/automl/efficientdet/inference.py", line 465, in build  *
        restore_ckpt(
    File "/home/gmpark/automl/efficientdet/inference.py", line 210, in restore_ckpt  *
        var_dict = ema.variables_to_restore(ema_vars)

    TypeError: Variable is unhashable. Instead, use variable.ref() as the key. (Variable: MirroredVariable:{
      0: <tf.Variable 'Variable:0' shape=() dtype=int64>,
      1: <tf.Variable 'Variable/replica_1:0' shape=() dtype=int64>
    